In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world");
world = root(mechanism)

RigidBodyDynamics.RigidBody{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),#undef)

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("b2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
x = MechanismState{Float64}(mechanism);

In [6]:
rand!(x)
x.v

DataStructures.OrderedDict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  RigidBodyDynamics.Joint… => [0.158463677519457,0.463096661181996,0.2364783579…
  RigidBodyDynamics.Joint… => [0.011709347222195854]

In [7]:
num_positions(x)

8

In [8]:
num_velocities(x)

7

In [9]:
cache = MechanismStateCache(mechanism, x);

RigidBodyDynamics.CartesianFrame3D("world")
RigidBodyDynamics.CartesianFrame3D("after_1")


In [10]:
relative_transform(cache, joint2.frameAfter, body1.frame)

RigidBodyDynamics.Transform3D{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("body1"),-0.5048660296793596 + 0.03909365416535249im + 0.832332421326275jm - 0.2253990210469865km,Vec(0.7061484165618157,0.7901757199413868,0.8643378328861966))

In [11]:
center_of_mass(mechanism, cache)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(-1.5484192641584578,-0.13504987781801508,0.5183564082108522))

In [12]:
mass(mechanism)

1.1893511259841671

In [13]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root(mechanism).frame)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(-2.133664683261013,-0.047423541535000946,0.30426593432733995))

In [14]:
relative_twist(cache, body2, body1)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("after_1"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.00259481462565872,0.002234077350776742,0.0025947686713973517),Vec(-0.0008028067523553895,0.006325879971874948,-0.004643716643850132))

In [15]:
relative_twist(cache, body1, world) + relative_twist(cache, body2, body1) + relative_twist(cache, world, body2)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.0,0.0,0.0),Vec(0.0,0.0,0.0))

In [16]:
geometric_jacobian(cache, path(mechanism, body2, world))

RigidBodyDynamics.GeometricJacobian{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("world"),6x7 Array{Float64,2}:
 -0.221602    0.431072   -0.108733   …  -0.0       -0.0       -0.0     
 -0.190794    0.21417    -0.952014      -0.0       -0.0       -0.0     
 -0.221598   -0.876532   -0.286087      -0.0       -0.0       -0.0     
  0.0685612   0.193933   -0.926849       0.431072  -0.108733   0.895742
 -0.540242   -0.532272    0.0679484      0.21417   -0.952014  -0.218632
  0.396582   -0.0346797   0.126155   …  -0.876532  -0.286087   0.387099)

In [17]:
mass_matrix(cache)

7x7 Array{Float64,2}:
 -0.479639   1.47605     0.0392167  …   0.192956  -0.34563      -0.29341  
 -1.32786    0.278205    2.49371        0.944363   0.200138      0.234618 
 -1.22736   -0.151919    0.416929       0.341176   0.444967     -0.0514921
  1.9917    -0.583459   -1.55083       -1.23319   -0.403857     -0.396097 
 -0.650944   1.37097    -0.0152261      0.0       -0.697445     -0.224315 
 -0.567676   0.0137359  -0.402161   …   0.697445  -2.77556e-17   0.0225968
 -0.29341    0.234618   -0.0514921     -0.224315   0.0225968     0.0910922

In [18]:
 A = momentum_matrix(cache)

RigidBodyDynamics.MomentumMatrix{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),6x7 Array{Float64,2}:
  0.0553404   0.916706   0.0846458  …   0.096617  -0.795929    0.0705281
 -0.789353   -0.476974   1.3037         0.84593    0.19427    -0.0570022
 -0.165653    1.23882    0.540454       0.254208  -0.343964   -0.0214509
 -0.420854    0.388279   1.02709       -0.093137   0.0982558   0.211186 
 -1.17038     1.43314    0.229719       0.416597  -0.577484    0.152273 
  1.77931    -0.124521  -1.20803    …  -1.35091   -0.553524    0.337394 )

In [19]:
A * velocity_vector(x)

RigidBodyDynamics.Momentum{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(-0.3664736896646964,0.40904543090692497,0.8649763748150394),Vec(0.29270633343452596,0.8271529817731434,-1.0311055375580331))